In [6]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import pickle
import napari
import numpy as np
import pandas as pd

from skimage.io import imread

import btrack
from btrack.constants import BayesianUpdates

sys.path.append('../libraries')
import input_functions as inp_f

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
info_file_path = r'D:\BARC\BARC_Wanjuan\210603_B3_5\210603_halo_B3_5_info.txt'

In [8]:
# read the file
info_file = open(info_file_path, 'r')
info_lines = info_file.readlines()
info_file.close()

# read info about the data frame
exp_dir,df_name = inp_f.read_df_info(info_lines)

df_dir = os.path.join(exp_dir,'df')
save_dir = df_dir

frames_to_exclude = inp_f.read_frames_2_exclude(info_lines)
frames_to_exclude = eval(frames_to_exclude)

modelPath = os.path.join(exp_dir,'code','libraries','cell_config.json')

## Read in the data frame objects data

In [9]:
data_df = pd.read_pickle(os.path.join(df_dir,df_name))

In [10]:
data_df.head()

,label,area,centroid-0,centroid-1,orientation,major_axis_length,minor_axis_length,bbox-0,bbox-1,bbox-2,bbox-3,image,mean_intensity-0_nuc,file,t,centroid-0_ring,centroid-1_ring,mean_intensity-0_ring,size_x,size_y
0,3,7552,62.466102,1571.076271,1.299775,115.218097,84.145963,20,1512,108,1628,"[[False, False, False, False, False, False, Fa...",596.054820,rpe c26 halo-qB3-5c2_000_label.tif,0,63.055952,1570.982447,321.047724,2048,2048
1,4,5584,98.107450,394.405444,1.478319,91.651491,77.862767,60,348,136,440,"[[False, False, False, False, False, False, Fa...",319.847421,rpe c26 halo-qB3-5c2_000_label.tif,0,98.053040,393.790427,277.563389,2048,2048
2,5,9568,117.586957,1926.155518,-0.692889,123.457777,98.921325,60,1872,176,1984,"[[False, False, False, False, False, False, Fa...",367.105456,rpe c26 halo-qB3-5c2_000_label.tif,0,117.509921,1926.253472,295.115575,2048,2048
3,6,15248,139.993179,731.732949,-1.339138,156.622341,124.390722,76,652,204,808,"[[False, False, False, False, False, False, Fa...",372.326731,rpe c26 halo-qB3-5c2_000_label.tif,0,140.406725,731.192554,288.602482,2048,2048
4,7,6464,150.123762,987.173267,-1.559444,108.356798,76.184376,112,932,188,1040,"[[False, False, False, False, False, False, Fa...",351.360922,rpe c26 halo-qB3-5c2_000_label.tif,0,150.217521,986.746126,295.524434,2048,2048


In [11]:
len(data_df)

14669

In [14]:
# create a structure suitable for tracking

# choose objects 
sel_vector = [not(x in frames_to_exclude) for x in data_df.t]

objects_gen = data_df.loc[sel_vector,['label','area','centroid-1','centroid-0','major_axis_length','minor_axis_length','t']]

objects_gen.columns=['ID', 'area', 'x', 'y', 'major_axis_length','minor_axis_length','t']
objects_gen['z']=0
objects_gen['label']=5
objects_gen['prob']=0
objects_gen['dummy']=False
objects_gen['states']=0

objects_gen.head()

,ID,area,x,y,major_axis_length,minor_axis_length,t,z,label,prob,dummy,states
0,3,7552,1571.076271,62.466102,115.218097,84.145963,0,0,5,0,False,0
1,4,5584,394.405444,98.107450,91.651491,77.862767,0,0,5,0,False,0
2,5,9568,1926.155518,117.586957,123.457777,98.921325,0,0,5,0,False,0
3,6,15248,731.732949,139.993179,156.622341,124.390722,0,0,5,0,False,0
4,7,6464,987.173267,150.123762,108.356798,76.184376,0,0,5,0,False,0


In [15]:
len(objects_gen)

14669

## Tracking proper

In [16]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file(modelPath)
    
    # approximate
    tracker.update_method = BayesianUpdates.APPROXIMATE
    tracker.max_search_radius = 100

    # append the objects to be tracked
    tracker.append(objects_gen)

    # set the volume (Z axis volume is set very large for 2D data)
    tracker.volume=((0, data_df.size_x[0]), (0, data_df.size_y[0]), (-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    # optional: get the data in a format for napari
    data, properties, graph = tracker.to_napari(ndim=2)
    # pickle Napari data
    with open(os.path.join(df_dir,'track.pkl'),'wb') as f:
        pickle.dump([data,properties,graph],f)

[INFO][2021/09/28 11:37:37 AM] Loaded btrack: C:\Users\kmkedz\AppData\Roaming\Python\Python37\site-packages\btrack\libs\libtracker.DLL
[INFO][2021/09/28 11:37:37 AM] btrack (v0.4.2) library imported
[INFO][2021/09/28 11:37:37 AM] Setting max XYZ search radius to: 100
[INFO][2021/09/28 11:37:37 AM] Starting BayesianTracker session
[INFO][2021/09/28 11:37:37 AM] Loading configuration file: D:\BARC\BARC_Wanjuan\210603_B3_5\code\libraries\cell_config.json
[INFO][2021/09/28 11:37:37 AM] Loading motion model: b'cell_motion'
[INFO][2021/09/28 11:37:37 AM] Setting Bayesian update method to: BayesianUpdates.APPROXIMATE
[INFO][2021/09/28 11:37:37 AM] Setting max XYZ search radius to: 100
[INFO][2021/09/28 11:37:37 AM] Objects are of type: <class 'pandas.core.frame.DataFrame'>
[INFO][2021/09/28 11:37:37 AM] Set volume to ((0, 2048), (0, 2048), (-100000.0, 100000.0))
[INFO][2021/09/28 11:37:37 AM] Starting tracking... 
[INFO][2021/09/28 11:37:37 AM] Tracking objects in frames 0 to 99 (of 221)...
[

## Merging objects and tracking information

In [17]:
trackDataAll = pd.DataFrame(data,columns=['track_id','t','x','y'])
trackDataAll['parent'] = properties['parent']
trackDataAll['generation'] = properties['generation']
trackDataAll['root'] = properties['root']

In [18]:
len(trackDataAll)

15158

In [19]:
trackDataAll

,track_id,t,x,y,parent,generation,root
0,1.0,0.0,1655.684012,892.015837,1,0,1
1,1.0,1.0,1649.548265,892.359729,1,0,1
2,2.0,0.0,956.946809,455.193617,2,0,2
3,2.0,1.0,945.201754,444.894737,2,0,2
4,3.0,0.0,992.019008,672.131405,3,0,3
...,...,...,...,...,...,...,...
15153,1164.0,218.0,207.614943,51.005747,1164,0,1164
15154,1164.0,219.0,203.036300,47.401639,1164,0,1164
15155,1164.0,220.0,199.056936,52.166667,1164,0,1164
15156,1166.0,217.0,148.151163,693.282946,1166,0,1166


In [20]:
allData = pd.merge(left=data_df,right=trackDataAll,left_on=['centroid-0','centroid-1','t'],right_on=['x','y','t'],how='left')

In [21]:
print(f'Number of all objects: {len(allData)}')

Number of all objects: 14669


In [22]:
allData.head()

,label,area,centroid-0,centroid-1,orientation,major_axis_length,minor_axis_length,bbox-0,bbox-1,bbox-2,...,centroid-1_ring,mean_intensity-0_ring,size_x,size_y,track_id,x,y,parent,generation,root
0,3,7552,62.466102,1571.076271,1.299775,115.218097,84.145963,20,1512,108,...,1570.982447,321.047724,2048,2048,27.0,62.466102,1571.076271,27.0,0.0,27.0
1,4,5584,98.107450,394.405444,1.478319,91.651491,77.862767,60,348,136,...,393.790427,277.563389,2048,2048,15.0,98.107450,394.405444,15.0,0.0,15.0
2,5,9568,117.586957,1926.155518,-0.692889,123.457777,98.921325,60,1872,176,...,1926.253472,295.115575,2048,2048,30.0,117.586957,1926.155518,30.0,0.0,30.0
3,6,15248,139.993179,731.732949,-1.339138,156.622341,124.390722,76,652,204,...,731.192554,288.602482,2048,2048,31.0,139.993179,731.732949,31.0,0.0,31.0
4,7,6464,150.123762,987.173267,-1.559444,108.356798,76.184376,112,932,188,...,986.746126,295.524434,2048,2048,32.0,150.123762,987.173267,32.0,0.0,32.0


## Define promising tracks

In [23]:
allData['accepted'] = False
allData['rejected'] = False
allData['promise'] = False

# mark tracks longer than 100 as promising
tracks_set = set(allData.track_id)

for track in tracks_set:
    
    track_len = np.sum(allData.track_id==track)
    
    if (track_len>50):
        
        allData.loc[allData.track_id==track,'promise'] = True

In [24]:
# save df
allData.to_pickle(os.path.join(df_dir,df_name))
allData.to_csv(os.path.join(df_dir,df_name.replace('pkl','csv')),index=False)